# Hyperparameter Tuning using Optuna

optuna라는 라이브러리를 이용하여 손쉽게 Hyperparameter를 최적화 할 수 있습니다. objective 함수에서 모델과 그 모델에 맞는 하이퍼 파라미터만 수정해주면 Xgboost가 아닌 다른 모델들도 실험해 볼 수 있습니다. 

### 데이터 및 라이브러리 로드

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_error
import optuna
from optuna import Trial
from sklearn import preprocessing
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [2]:
train = pd.read_csv('../input/pceo-ai-club1/train.csv', index_col="student_id") #반드시 index_col을 student_id로 설정해주세요
test = pd.read_csv('../input/pceo-ai-club1/test.csv', index_col="student_id")
sample_submission = pd.read_csv('../input/pceo-ai-club1/sample_submission.csv', index_col="student_id")

### Super Simple EDA

In [3]:
train.head()

,school,school_setting,school_type,classroom,teaching_method,n_student,gender,lunch,pretest
student_id,,,,,,,,,
W20C6,GOOBU,Urban,Public,HKF,Standard,28.0,Female,Qualifies for reduced/free lunch,34.0
A3JJP,GOOBU,Urban,Public,W8A,Experimental,26.0,Female,Qualifies for reduced/free lunch,36.0
UBDCF,DNQDD,Suburban,Public,PW5,Experimental,20.0,Male,Qualifies for reduced/free lunch,42.0
QQJEC,GOOBU,Urban,Public,W8A,Experimental,26.0,Female,Qualifies for reduced/free lunch,36.0
3SWL5,UUUQX,Suburban,Non-public,SSP,Standard,15.0,Female,Does not qualify,66.0


pretest 값이 우리가 예측해야 하는 시험 점수입니다.

In [4]:
test.head()

,school,school_setting,school_type,classroom,teaching_method,n_student,gender,lunch
student_id,,,,,,,,
MNVG5,VHDHF,Rural,Non-public,KR1,Experimental,15.0,Female,Qualifies for reduced/free lunch
LB2TH,GJJHK,Suburban,Public,ZDT,Standard,27.0,Male,Qualifies for reduced/free lunch
ETDRG,CIMBB,Urban,Non-public,PC6,Standard,17.0,Female,Does not qualify
TME5H,GOKXL,Rural,Public,VA6,Standard,19.0,Female,Qualifies for reduced/free lunch
G8C0U,QOQTS,Urban,Public,3XJ,Standard,24.0,Female,Does not qualify


In [5]:
train.isnull().any()

school             False
school_setting     False
school_type        False
classroom          False
teaching_method    False
n_student          False
gender             False
lunch              False
pretest            False
dtype: bool

위에서 보다시피 데이터에 null 값은 없습니다.

### Data Preprocessing

In [6]:
y = train.pretest
X = train.drop(["pretest"], axis=1)

In [7]:
s = (X.dtypes == 'object')
object_cols = list(s[s].index)
object_cols

['school',
 'school_setting',
 'school_type',
 'classroom',
 'teaching_method',
 'gender',
 'lunch']

n_students 이외의 feature들은 모두 categorical입니다. (cateogical => 문자 형태)

In [8]:
label_X = X.copy()
label_test = test.copy()

ordinal_encoder = OrdinalEncoder()
label_X[object_cols] = ordinal_encoder.fit_transform(X[object_cols])
label_test[object_cols] = ordinal_encoder.transform(test[object_cols])

In [9]:
label_X.head()

,school,school_setting,school_type,classroom,teaching_method,n_student,gender,lunch
student_id,,,,,,,,
W20C6,8.0,2.0,1.0,45.0,1.0,28.0,0.0,1.0
A3JJP,8.0,2.0,1.0,83.0,0.0,26.0,0.0,1.0
UBDCF,4.0,1.0,1.0,65.0,0.0,20.0,1.0,1.0
QQJEC,8.0,2.0,1.0,83.0,0.0,26.0,0.0,1.0
3SWL5,17.0,1.0,0.0,73.0,1.0,15.0,0.0,0.0


In [10]:
label_test.head()

,school,school_setting,school_type,classroom,teaching_method,n_student,gender,lunch
student_id,,,,,,,,
MNVG5,18.0,0.0,0.0,53.0,0.0,15.0,0.0,1.0
LB2TH,6.0,1.0,1.0,95.0,1.0,27.0,1.0,1.0
ETDRG,2.0,2.0,0.0,62.0,1.0,17.0,0.0,0.0
TME5H,7.0,0.0,1.0,82.0,1.0,19.0,0.0,1.0
G8C0U,14.0,2.0,1.0,15.0,1.0,24.0,0.0,0.0


categorical variable을 숫자로 잘 바꾸어 주었습니다. Ordinal Encoder에 대해 잊어버린 분들은 [kaggle course](https://www.kaggle.com/alexisbcook/categorical-variables)를 참고하세요.

In [11]:
X_train, X_val, y_train, y_val = train_test_split(label_X, y, test_size=0.2, random_state=3)

### Model train

In [12]:
def objective(trial):
    # To select which parameters to optimize, please look at the XGBoost documentation:
    # https://xgboost.readthedocs.io/en/latest/parameter.html
    param = {
        'tree_method':'gpu_hist',  # Use GPU acceleration
        'lambda': trial.suggest_loguniform(
            'lambda', 1e-3, 10.0
        ),
        'alpha': trial.suggest_loguniform(
            'alpha', 1e-3, 10.0
        ),
        'colsample_bytree': trial.suggest_categorical(
            'colsample_bytree', [0.5,0.6,0.7,0.8,0.9,1.0]
        ),
        'subsample': trial.suggest_categorical(
            'subsample', [0.6,0.7,0.8,1.0]
        ),
        'learning_rate': trial.suggest_categorical(
            'learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]
        ),
        'n_estimators': trial.suggest_categorical(
            "n_estimators", [150, 200, 300, 3000]
        ),
        'max_depth': trial.suggest_categorical(
            'max_depth', [4,5,7,9,11,13,15,17]
        ),
        'random_state': 42,
        'min_child_weight': trial.suggest_int(
            'min_child_weight', 1, 300
        ),
    }
    model = XGBRegressor(**param)  
    
    model.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(X_val)
    
    mae = mean_absolute_error(y_val, preds)
    
    return mae

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-31 12:52:20,249] A new study created in memory with name: no-name-1cba9e61-ded8-4668-b6bc-7f051239198d
[I 2021-08-31 12:52:21,274] Trial 0 finished with value: 14.664788603085524 and parameters: {'lambda': 0.1349057779749931, 'alpha': 0.1587853100528135, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.009, 'n_estimators': 150, 'max_depth': 17, 'min_child_weight': 140}. Best is trial 0 with value: 14.664788603085524.
[I 2021-08-31 12:52:21,489] Trial 1 finished with value: 11.339020143475448 and parameters: {'lambda': 0.03454243008365202, 'alpha': 0.049348395396790944, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.009, 'n_estimators': 200, 'max_depth': 11, 'min_child_weight': 236}. Best is trial 1 with value: 11.339020143475448.
[I 2021-08-31 12:52:26,151] Trial 2 finished with value: 4.286704280920196 and parameters: {'lambda': 0.007430692042043475, 'alpha': 0.004352470294815615, 'colsample_bytree': 0.9, 'subsample': 0.6, 'learning_rate': 0.018,

Number of finished trials: 200
Best trial: {'lambda': 0.004508685436604593, 'alpha': 0.06418004670018108, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.016, 'n_estimators': 3000, 'max_depth': 7, 'min_child_weight': 39}


In [14]:
def lgbm_objective(trial):
    # To select which parameters to optimize, please look at the XGBoost documentation:
    # https://xgboost.readthedocs.io/en/latest/parameter.html
    param = {
        "objective": "regression",
        "metric": "mae",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    model = LGBMRegressor(**param)  
    
    model.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(X_val)
    
    mae = mean_absolute_error(y_val, preds)
    
    return mae

In [15]:
def cat_objective(trial):
    # To select which parameters to optimize, please look at the XGBoost documentation:
    # https://xgboost.readthedocs.io/en/latest/parameter.html
    param = {
        "objective": "regression",
        "metric": "mae",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    model = LGBMRegressor(**param)  
    
    model.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(X_val)
    
    mae = mean_absolute_error(y_val, preds)
    
    return mae

In [16]:
study = optuna.create_study(direction='minimize')
study.optimize(lgbm_objective, n_trials=200)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-31 13:06:59,069] A new study created in memory with name: no-name-86bacc3f-74c1-4c0c-a319-f835d718cf1c
[I 2021-08-31 13:06:59,141] Trial 0 finished with value: 4.927542784768548 and parameters: {'lambda_l1': 1.713692109696966e-05, 'lambda_l2': 3.390059245630015e-05, 'num_leaves': 107, 'feature_fraction': 0.733551333144171, 'bagging_fraction': 0.41301446174134276, 'bagging_freq': 6, 'min_child_samples': 81}. Best is trial 0 with value: 4.927542784768548.
[I 2021-08-31 13:06:59,258] Trial 1 finished with value: 2.719098097331571 and parameters: {'lambda_l1': 1.5557229533441639e-07, 'lambda_l2': 0.00938116890634378, 'num_leaves': 54, 'feature_fraction': 0.9017024132598177, 'bagging_fraction': 0.4794759334591751, 'bagging_freq': 3, 'min_child_samples': 12}. Best is trial 1 with value: 2.719098097331571.


[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.733551333144171, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.733551333144171
[LightGBM] [Warning] bagging_fraction is set=0.41301446174134276, subsample=1.0 will be ignored. Current value: bagging_fraction=0.41301446174134276
[LightGBM] [Warning] lambda_l2 is set=3.390059245630015e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.390059245630015e-05
[LightGBM] [Warning] lambda_l1 is set=1.713692109696966e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.713692109696966e-05
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.9017024132598177, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9017024132598177
[LightGBM] [Warning] bagging_fraction is set=0.479

[I 2021-08-31 13:06:59,386] Trial 2 finished with value: 3.1207364020388333 and parameters: {'lambda_l1': 0.0002260432278411291, 'lambda_l2': 0.0035314251879959733, 'num_leaves': 92, 'feature_fraction': 0.4136834582467992, 'bagging_fraction': 0.8228340952447666, 'bagging_freq': 7, 'min_child_samples': 62}. Best is trial 1 with value: 2.719098097331571.
[I 2021-08-31 13:06:59,542] Trial 3 finished with value: 2.7897847984952007 and parameters: {'lambda_l1': 7.441763681411817e-08, 'lambda_l2': 7.673378809821379e-06, 'num_leaves': 214, 'feature_fraction': 0.5665010727765203, 'bagging_fraction': 0.9777241790881985, 'bagging_freq': 7, 'min_child_samples': 56}. Best is trial 1 with value: 2.719098097331571.


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.5665010727765203, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5665010727765203
[LightGBM] [Warning] bagging_fraction is set=0.9777241790881985, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9777241790881985
[LightGBM] [Warning] lambda_l2 is set=7.673378809821379e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.673378809821379e-06
[LightGBM] [Warning] lambda_l1 is set=7.441763681411817e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.441763681411817e-08
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.9945996665481539, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9945996665481539
[LightGBM] [Warning] bagging_fraction is set=0.536

[I 2021-08-31 13:06:59,656] Trial 4 finished with value: 4.180011327482156 and parameters: {'lambda_l1': 0.0002425137781892736, 'lambda_l2': 2.320697837884449e-08, 'num_leaves': 44, 'feature_fraction': 0.9945996665481539, 'bagging_fraction': 0.5360576252977897, 'bagging_freq': 1, 'min_child_samples': 86}. Best is trial 1 with value: 2.719098097331571.
[I 2021-08-31 13:06:59,784] Trial 5 finished with value: 4.0198324587294305 and parameters: {'lambda_l1': 7.018293193686061e-07, 'lambda_l2': 2.766442582783176e-07, 'num_leaves': 5, 'feature_fraction': 0.9951817854901577, 'bagging_fraction': 0.5908059464862193, 'bagging_freq': 4, 'min_child_samples': 47}. Best is trial 1 with value: 2.719098097331571.


...

[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.4383942351879003, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4383942351879003
[LightGBM] [Warning] bagging_fraction is set=0.9777184194627394, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9777184194627394
[LightGBM] [Warning] lambda_l2 is set=0.00011469311507503528, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00011469311507503528
[LightGBM] [Warning] lambda_l1 is set=1.0194487157622283, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0194487157622283
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.40856619956401385, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.40856619956401385
[LightGBM] [Warning] bagging_fraction is set=0.99974

[I 2021-08-31 13:07:25,991] Trial 196 finished with value: 2.5707288346005637 and parameters: {'lambda_l1': 1.4411060633094086, 'lambda_l2': 4.183540493107634e-05, 'num_leaves': 158, 'feature_fraction': 0.40856619956401385, 'bagging_fraction': 0.9997431009669151, 'bagging_freq': 5, 'min_child_samples': 11}. Best is trial 129 with value: 2.531933697822227.
[I 2021-08-31 13:07:26,143] Trial 197 finished with value: 2.6069739467521087 and parameters: {'lambda_l1': 0.6382808006375383, 'lambda_l2': 0.00030087938527746894, 'num_leaves': 138, 'feature_fraction': 0.45327526655975114, 'bagging_fraction': 0.9886411474955034, 'bagging_freq': 5, 'min_child_samples': 9}. Best is trial 129 with value: 2.531933697822227.


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.45327526655975114, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45327526655975114
[LightGBM] [Warning] bagging_fraction is set=0.9886411474955034, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9886411474955034
[LightGBM] [Warning] lambda_l2 is set=0.00030087938527746894, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00030087938527746894
[LightGBM] [Warning] lambda_l1 is set=0.6382808006375383, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6382808006375383
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.43516216847145367, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43516216847145367
[LightGBM] [Warning] bagging_fraction is set=0.956

[I 2021-08-31 13:07:26,285] Trial 198 finished with value: 2.547136890595558 and parameters: {'lambda_l1': 0.3219087448765091, 'lambda_l2': 0.00016365603658042609, 'num_leaves': 150, 'feature_fraction': 0.43516216847145367, 'bagging_fraction': 0.956572175862579, 'bagging_freq': 5, 'min_child_samples': 7}. Best is trial 129 with value: 2.531933697822227.
[I 2021-08-31 13:07:26,423] Trial 199 finished with value: 2.532167917942557 and parameters: {'lambda_l1': 0.8384571815144879, 'lambda_l2': 0.00025029694554027244, 'num_leaves': 169, 'feature_fraction': 0.419097261868845, 'bagging_fraction': 0.974744879804673, 'bagging_freq': 5, 'min_child_samples': 5}. Best is trial 129 with value: 2.531933697822227.


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.419097261868845, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.419097261868845
[LightGBM] [Warning] bagging_fraction is set=0.974744879804673, subsample=1.0 will be ignored. Current value: bagging_fraction=0.974744879804673
[LightGBM] [Warning] lambda_l2 is set=0.00025029694554027244, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00025029694554027244
[LightGBM] [Warning] lambda_l1 is set=0.8384571815144879, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8384571815144879
Number of finished trials: 200
Best trial: {'lambda_l1': 1.6221791729724848, 'lambda_l2': 1.30720522088146e-06, 'num_leaves': 182, 'feature_fraction': 0.43283266317298674, 'bagging_fraction': 0.9795305379659173, 'bagging_freq': 5, 'min_child_samples': 5}


In [17]:
best_params = {'lambda': 0.005741219280777098, 
               'alpha': 2.1759591892131596, 
               'colsample_bytree': 0.7, 
               'subsample': 0.7, 
               'learning_rate': 0.02, 
               'n_estimators': 3000, 
               'max_depth': 17, 
               'min_child_weight': 49}
model = XGBRegressor(**best_params)  
    
model.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)

preds = model.predict(X_val)

mae = mean_absolute_error(y_val, preds)

print(mae)

2.6143834242346693


In [18]:
lgbm_params = {'lambda_l1': 0.031412754431276696, 
               'lambda_l2': 2.0633166117061854e-06, 
               'num_leaves': 163, 
               'feature_fraction': 0.42972618195865353, 
               'bagging_fraction': 0.7334168518029209, 
               'bagging_freq': 3, 
               'min_child_samples': 5}

model = LGBMRegressor(**lgbm_params)  
    
model.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)

preds = model.predict(X_val)

mae = mean_absolute_error(y_val, preds)

print(mae)

[LightGBM] [Warning] feature_fraction is set=0.42972618195865353, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42972618195865353
[LightGBM] [Warning] lambda_l2 is set=2.0633166117061854e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.0633166117061854e-06
[LightGBM] [Warning] lambda_l1 is set=0.031412754431276696, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.031412754431276696
[LightGBM] [Warning] bagging_fraction is set=0.7334168518029209, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7334168518029209
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
2.5257972584550665


### Model predict & submission

In [19]:
result = model.predict(label_test)

In [20]:
submission = pd.DataFrame(result,columns=["pretest"], index=label_test.index)

In [21]:
submission.to_csv("submission_optuna_lgbm.csv")